In [ ]:
import pandas as pd

# 1. 파일 읽기 (구분자를 ';'로 설정)
file_path = "data/kbo_regular_seasons_result.csv"

try:
    # sep=';' 를 통해 세미콜론 구분자 파일 읽기
    df = pd.read_csv(file_path, sep=';', encoding='utf-8')
except Exception:
    df = pd.read_csv(file_path, sep=';', encoding='cp949')

# 2. 구단 계보 ID 매핑 (TeamID)
team_lineage = {
    '삼성 라이온즈': 'SAM', '롯데 자이언츠': 'LOT',
    'OB 베어스': 'DOO', '두산 베어스': 'DOO',
    'MBC 청룡': 'LGT', 'LG 트윈스': 'LGT',
    '해태 타이거즈': 'KIA', 'KIA 타이거즈': 'KIA',
    '빙그레 이글스': 'HAN', '한화 이글스': 'HAN',
    '삼미 슈퍼스타즈': 'HYU', '청보 핀토스': 'HYU', '태평양 돌핀스': 'HYU', '현대 유니콘스': 'HYU',
    'SK 와이번스': 'SSG', 'SSG 랜더스': 'SSG',
    '우리 히어로즈': 'HER', '히어로즈': 'HER', '넥센 히어로즈': 'HER', '키움 히어로즈': 'HER',
    'NC 다이노스': 'NCD', 'KT 위즈': 'KTW', '쌍방울 레이더스': 'SBB'
}
df['TeamID'] = df['Team'].map(team_lineage)

# 3. 데이터 타입 숫자 변환 (에러 방지)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df['WPCT'] = pd.to_numeric(df['WPCT'], errors='coerce')
df['ERA'] = pd.to_numeric(df['ERA'], errors='coerce')
df = df.dropna(subset=['Year', 'WPCT']).copy() # 유효하지 않은 데이터 제거

# 4. 연도별 순위 계산 (WPCT 내림차순, 동일 시 ERA 오름차순)
# 승률이 같을 경우 평균자책점(ERA)이 낮은 팀이 더 높은 순위가 되도록 설정
df = df.sort_values(by=['Year', 'WPCT', 'ERA'], ascending=[True, False, True])
df['Rank'] = df.groupby('Year').cumcount() + 1

# 5. 결과 확인 및 저장
# 컬럼 순서를 보기 좋게 재배치 (Year, Rank, TeamID 순)
cols = ['Year', 'Rank', 'TeamID', 'Team', 'Games', 'W', 'L', 'D', 'BA', 'ERA', 'WPCT']
df = df[cols]

print(df.head(10))

# 최종 결과 저장 (CSV 저장 시에는 쉼표 구분자가 표준이지만, 필요시 다시 세미콜론 설정 가능)
df.to_csv('KBO_final_ranked_data.csv', index=False, encoding='utf-8-sig')

--- 연도별 순위 및 TeamID 부여 결과 (상위 10개) ---
   Year  Rank TeamID      Team  Games   W   L  D   BA   ERA  WPCT
0  1982     1    DOO    OB 베어스     80  56  24  0  283  3.20   700
1  1982     2    SAM   삼성 라이온즈     80  54  26  0  266  2.70   675
2  1982     3    LGT    MBC 청룡     80  46  34  0  282  3.51   575
3  1982     4    KIA   해태 타이거즈     80  38  42  0  261  3.79   475
4  1982     5    LOT   롯데 자이언츠     80  31  49  0  256  3.95   388
5  1982     6    HYU  삼미 슈퍼스타즈     80  15  65  0  240  6.23   188
7  1983     1    LGT    MBC 청룡    100  55  43  2  255  2.71   561
6  1983     2    KIA   해태 타이거즈    100  55  44  1  268  3.17   556
8  1983     3    HYU  삼미 슈퍼스타즈    100  52  47  1  245  3.04   525
9  1983     4    SAM   삼성 라이온즈    100  46  50  4  263  3.42   479


In [ ]:
import pandas as pd
import os


reg_path = "KBO_final_ranked_data.csv"
ps_path = "data/postseasons_result.csv"

def load_kbo_csv(path):
    if not os.path.exists(path):
        print(f"파일 없음: {path}")
        return None
    
    df = pd.read_csv(path, sep=';', encoding='utf-8-sig')
    if len(df.columns) <= 1:
        df = pd.read_csv(path, sep=',', encoding='utf-8-sig')
  
    df.columns = df.columns.str.strip()
    return df

df_regular = load_kbo_csv(reg_path)
df_ps = load_kbo_csv(ps_path)

if df_regular is not None and df_ps is not None:
    print(df_regular.columns.tolist())
    print(df_ps.columns.tolist())

    try:
        df_total = pd.merge(df_regular, df_ps, on=['Year', 'TeamID'], how='left')
        
        # 가을야구 미진출 팀(NaN)은 0으로 채우기
        df_total['PS'] = df_total['PS'].fillna(0).astype(int)
        
        print(df_total[['Year', 'TeamID', 'Rank', 'PS']].head(10))
        
        df_total.to_csv('kbo_total_data_final.csv', index=False, encoding='utf-8-sig')
        print("\n'kbo_total_data_final.csv' ")
        
    except KeyError as e:
        print(e)

['Year', 'Rank', 'TeamID', 'Team', 'Games', 'W', 'L', 'D', 'BA', 'ERA', 'WPCT']
['Year', 'TeamID', 'PS']
   Year TeamID  Rank  PS
0  1982    DOO     1   1
1  1982    SAM     2   2
2  1982    LGT     3   0
3  1982    KIA     4   0
4  1982    LOT     5   0
5  1982    HYU     6   0
6  1983    LGT     1   2
7  1983    KIA     2   1
8  1983    HYU     3   0
9  1983    SAM     4   0

'kbo_total_data_final.csv' 


In [ ]:
import pandas as pd
import os

path = "kbo_total_data_final.csv"
df = pd.read_csv(path, sep=',', encoding='utf-8-sig')


# PS 컬럼 1 = 5, 2 = 4, 3 = 3, 4 = 2, 5 = 1, 나머지 0
ps_points = {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
df['PS'] = df['PS'].map(ps_points).fillna(0).astype(int)  

print(df.head(10))



--- 최종 데이터 상위 10개 ---
   Year  Rank TeamID      Team  Games   W   L  D   BA   ERA  WPCT  PS
0  1982     1    DOO    OB 베어스     80  56  24  0  283  3.20   700   5
1  1982     2    SAM   삼성 라이온즈     80  54  26  0  266  2.70   675   4
2  1982     3    LGT    MBC 청룡     80  46  34  0  282  3.51   575   0
3  1982     4    KIA   해태 타이거즈     80  38  42  0  261  3.79   475   0
4  1982     5    LOT   롯데 자이언츠     80  31  49  0  256  3.95   388   0
5  1982     6    HYU  삼미 슈퍼스타즈     80  15  65  0  240  6.23   188   0
6  1983     1    LGT    MBC 청룡    100  55  43  2  255  2.71   561   4
7  1983     2    KIA   해태 타이거즈    100  55  44  1  268  3.17   556   5
8  1983     3    HYU  삼미 슈퍼스타즈    100  52  47  1  245  3.04   525   0
9  1983     4    SAM   삼성 라이온즈    100  46  50  4  263  3.42   479   0


In [28]:
file_path = "data/kbo_regular_seasons_result.csv"
df = pd.read_csv(file_path)

print(df.head())

   Year     Team  Games   W   L  D   BA   ERA  WPCT
0  1982   OB 베어스     80  56  24  0  283  3.20   700
1  1982  삼성 라이온즈     80  54  26  0  266  2.70   675
2  1982   MBC 청룡     80  46  34  0  282  3.51   575
3  1982  해태 타이거즈     80  38  42  0  261  3.79   475
4  1982  롯데 자이언츠     80  31  49  0  256  3.95   388
